In [1]:
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
import pandas as pd
import numpy as np

In [2]:
pd.read_csv('C:/Users/DELL/Documents/workspace/reto502/reto_502_api/ts-forecasting-pipeline/data/01_raw/dataquery.csv', index_col=0)

,Fecha_hoy,ID_empresa,ing_hab
0,2021-11-15,1,1.165129e+06
1,2021-11-15,6,2.150826e+05
2,2021-11-15,2,1.272147e+06
3,2021-11-15,4,2.581829e+04
4,2021-11-15,3,2.034338e+05
...,...,...,...
8604,2024-11-02,1,1.101201e+06
8605,2024-11-02,5,3.831494e+04
8606,2024-11-02,6,1.478103e+05
8607,2024-11-02,3,8.760099e+04


In [3]:
# load data
def load_data() -> pd.DataFrame:
    '''
    Args:
        None

    Returns:
        pivot_data [pd.DataFrame]: Dataframe containing time series by company_id.
    '''
    data = pd.read_csv('C:/Users/DELL/Documents/workspace/reto502/reto_502_api/ts-forecasting-pipeline/data/01_raw/dataquery.csv', index_col=0)
    data.columns = ['date', 'company', 'data']
    data['date'] = pd.to_datetime(data['date'])

    pivot_data = data.pivot_table(index='date', columns='company', values='data', aggfunc='sum')
    pivot_data.columns = [f'company_{col}' for col in pivot_data.columns]
    pivot_data.reset_index(inplace=True)

    return pivot_data

pivot_data = load_data()
pivot_data

,date,company_1,company_2,company_3,company_4,company_5,company_6
0,2020-11-01,9.002395e+05,NaN,2.740852e+04,NaN,NaN,NaN
1,2020-11-02,7.580464e+05,NaN,1.102843e+05,NaN,NaN,NaN
2,2020-11-03,7.018318e+05,NaN,1.185705e+05,NaN,NaN,NaN
3,2020-11-04,5.149224e+05,NaN,1.210628e+05,NaN,NaN,NaN
4,2020-11-05,7.544703e+05,NaN,1.639952e+05,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1516,2024-12-27,2.200914e+06,2.594027e+06,1.213341e+06,6978.373338,50096.2245,653910.662472
1517,2024-12-28,2.312137e+06,2.603804e+06,1.103839e+06,6675.982785,35172.2700,665208.517454
1518,2024-12-29,2.174856e+06,2.783804e+06,1.272284e+06,6604.603972,32966.6400,587478.563432
1519,2024-12-30,2.713556e+06,2.780263e+06,1.378832e+06,6360.479553,32294.2095,660170.179135


In [4]:
def fill_missing_dates(data:pd.DataFrame) -> pd.DataFrame:
    '''
    Creates empty rows for dates that are missing in the time series.
    
    Parameters:
        data (pd.DataFrame): DataFrame containing the date column and a columns for each company ID.
    
    Returns:
        pd.DataFrame: DataFrame with missing dates filled with null values.
    '''

    date_column = 'date'
    
    # Ensure the date column is in datetime format
    data[date_column] = pd.to_datetime(data[date_column])
    
    # Set the date column as the index
    data.set_index(date_column, inplace=True)
    
    # Create a complete date range from the min to the max date in the DataFrame
    complete_date_range = pd.date_range(start=data.index.min(), end=data.index.max(), freq='D')
    
    # Reindex the DataFrame to this complete date range
    df_reindexed = data.reindex(complete_date_range)
    
    # Rename the index back to the original date column name
    # df_reindexed.index.name = date_column
    
    # Reset index to convert the date index back to a column
    # df_reindexed.reset_index(inplace=True)
    
    return df_reindexed

complete_dates = pivot_data.copy()
complete_dates = fill_missing_dates(pivot_data)
complete_dates

,company_1,company_2,company_3,company_4,company_5,company_6
2020-11-01,9.002395e+05,NaN,2.740852e+04,NaN,NaN,NaN
2020-11-02,7.580464e+05,NaN,1.102843e+05,NaN,NaN,NaN
2020-11-03,7.018318e+05,NaN,1.185705e+05,NaN,NaN,NaN
2020-11-04,5.149224e+05,NaN,1.210628e+05,NaN,NaN,NaN
2020-11-05,7.544703e+05,NaN,1.639952e+05,NaN,NaN,NaN
...,...,...,...,...,...,...
2024-12-27,2.200914e+06,2.594027e+06,1.213341e+06,6978.373338,50096.2245,653910.662472
2024-12-28,2.312137e+06,2.603804e+06,1.103839e+06,6675.982785,35172.2700,665208.517454
2024-12-29,2.174856e+06,2.783804e+06,1.272284e+06,6604.603972,32966.6400,587478.563432
2024-12-30,2.713556e+06,2.780263e+06,1.378832e+06,6360.479553,32294.2095,660170.179135


In [5]:
def handle_outliers(data: pd.DataFrame) -> pd.DataFrame:
    '''
    Identify and handle outliers.
    Replaces outliers by maximum or minimum value calculated by a distance of 3
    standard deviations from the mean.

    Args:
        dataframe [pd.DataFrame]: Columns for each company ID as independent 
                                  time series, date index.


    Returns:
        [pd.DataFrame]: Same columns, outliers replaced by NaN values.


    '''
    data_no_outliers = []
    for company in data.columns:
        data_by_company = data[company]
        mean = np.mean(data_by_company)
        std_dev = np.std(data_by_company)
        lower_limit = mean - 3 * std_dev
        upper_limit = mean + 3 * std_dev

        data_capped = np.clip(data_by_company, lower_limit, upper_limit)
        data_no_outliers.append(data_capped)

    data_no_outliers = pd.concat(data_no_outliers, axis=1, keys=data.columns)
    data_no_outliers

    return data_no_outliers

no_outliers = complete_dates.copy()
no_outliers = handle_outliers(no_outliers)
no_outliers

,company_1,company_2,company_3,company_4,company_5,company_6
2020-11-01,9.002395e+05,NaN,2.740852e+04,NaN,NaN,NaN
2020-11-02,7.580464e+05,NaN,1.102843e+05,NaN,NaN,NaN
2020-11-03,7.018318e+05,NaN,1.185705e+05,NaN,NaN,NaN
2020-11-04,5.149224e+05,NaN,1.210628e+05,NaN,NaN,NaN
2020-11-05,7.544703e+05,NaN,1.639952e+05,NaN,NaN,NaN
...,...,...,...,...,...,...
2024-12-27,2.200914e+06,2.594027e+06,1.213341e+06,6978.373338,49786.120225,644244.202936
2024-12-28,2.312137e+06,2.603804e+06,1.103839e+06,6675.982785,35172.270000,644244.202936
2024-12-29,2.174856e+06,2.759007e+06,1.257331e+06,6604.603972,32966.640000,587478.563432
2024-12-30,2.478231e+06,2.759007e+06,1.257331e+06,6360.479553,32294.209500,644244.202936


In [6]:
def fill_missing_data(df: pd.DataFrame) -> pd.DataFrame:
  '''
  Fill missing data for each company's time series.

  Args:
      dataframe [pd.DataFrame]: Columns for each company ID as independent 
                                time series, date index.
  Returns:
      [pd.DataFrame]: Same columns, missing data filled.
    '''
  for column in df.columns:
    df_revenue = df[column].to_frame()
    decomposition = seasonal_decompose(df[column].dropna(), period=365)
    df_revenue['trend'] = decomposition._trend
    df_revenue['seasonal'] = decomposition._seasonal
    df_revenue['residual'] = decomposition._resid
    df_revenue['month'] = df_revenue.index.month
    df_revenue['day'] = df_revenue.index.day

    revenue = df[column]
    left = 0
    start = 0
    for i in range(len(revenue)):
      right = i
      if not np.isnan(revenue.iloc[right]) and not np.isnan(revenue.iloc[left]) and np.isnan(revenue.iloc[right - 1]):
          df_revenue.loc[df_revenue.index[left-10:right+10], 'trend'] = df_revenue.loc[df_revenue.index[left-10:right+10], 'trend'].interpolate(method='spline', order=3, limit_direction='both')

          # Calculate residuals and seasonal means
          residuals = df_revenue.groupby(['month', 'day'])['residual'].mean()
          seasonal = df_revenue.groupby(['month', 'day'])['seasonal'].mean()

          # Extract the range of residuals and seasonal values
          start_month_day = (df_revenue['month'].iloc[left+1], df_revenue['day'].iloc[left+1])
          end_month_day = (df_revenue['month'].iloc[right-1], df_revenue['day'].iloc[right-1])

          # Slicing by index from the grouped means
          residual_vals = residuals.loc[start_month_day:end_month_day].values
          seasonal_vals = seasonal.loc[start_month_day:end_month_day].values

          # Calculate new values for the nulls
          trend_vals = df_revenue['trend'].iloc[left+1:right].values
          nulos = residual_vals + seasonal_vals + trend_vals

          revenue.iloc[left+1:right] = nulos
      if left == 0 and not np.isnan(revenue.iloc[right]):
        start = right
      if (not np.isnan(revenue.iloc[right]) and np.isnan(revenue.iloc[left])) or not np.isnan(revenue.iloc[right]):
        left = right

    persistent_nulls = revenue[start:][revenue[start:].isnull()].index.tolist()

    for indexes in persistent_nulls:
      revenue[indexes] = (revenue[indexes + pd.Timedelta(days=1)] + revenue[indexes - pd.Timedelta(days=1)])/2

    df[column] = revenue
  return df


no_missing_values = no_outliers.copy()
no_missing_values = fill_missing_data(no_missing_values)
no_missing_values

,company_1,company_2,company_3,company_4,company_5,company_6
2020-11-01,9.002395e+05,NaN,2.740852e+04,NaN,NaN,NaN
2020-11-02,7.580464e+05,NaN,1.102843e+05,NaN,NaN,NaN
2020-11-03,7.018318e+05,NaN,1.185705e+05,NaN,NaN,NaN
2020-11-04,5.149224e+05,NaN,1.210628e+05,NaN,NaN,NaN
2020-11-05,7.544703e+05,NaN,1.639952e+05,NaN,NaN,NaN
...,...,...,...,...,...,...
2024-12-27,2.200914e+06,2.594027e+06,1.213341e+06,6978.373338,49786.120225,644244.202936
2024-12-28,2.312137e+06,2.603804e+06,1.103839e+06,6675.982785,35172.270000,644244.202936
2024-12-29,2.174856e+06,2.759007e+06,1.257331e+06,6604.603972,32966.640000,587478.563432
2024-12-30,2.478231e+06,2.759007e+06,1.257331e+06,6360.479553,32294.209500,644244.202936
